<a id="top"></a>
# Xarray and Gridded Data

<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" width=200/>

Xarray builds upon pandas to provide labeled multi-dimensional arrays. Xarray datasets and data arrays can be manipulated similarly to NumPy arrays and pandas Dataframes, while providing additional tools specific to gridded datasets.

1. [DataArrays](#data-arrays)
2. [Datasets](#datasets)
3. [Working with real data](#real-data)
4. [Scipy for spatial analysis](#scipy)

## Exercises

[Exercise 1](#exercise1)<br>
[Exercise 2](#exercise2)

<a id="data-arrays"></a>
## 1. DataArrays

Like pandas Series objects, xarray has objects called DataArrays that store labeled data points. Xarray expands on the labeled data model by expanding to multiple dimensions (i.e., gridded data) and providing more options for metadata descriptions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
coords = {
    "Time": pd.date_range("2020-01-01 00:00", "2020-01-02 00:00", freq="H", name="Time"),
}
data = np.sin(np.arange(0.0, coords["Time"].size * 0.5, 0.5))

sinx = xr.DataArray(data=data, coords=coords, dims=["Time"], name="sinx")

In [ ]:
sinx

In [ ]:
sinx.Time

In [ ]:
sinx.data

In [ ]:
sinx.coords

[Return to top](#top)

<a id="datasets"></a>
## 2. Datasets

Datasets typically contain multiple DataArrays with some common coordinates. Let's make a dataset out of sinx and cosx.

In [ ]:
coords = {
    "Time": pd.date_range("2020-01-01 00:00", "2020-01-02 00:00", freq="H", name="Time"),
}
sinx = np.sin(np.arange(0.0, coords["Time"].size * 0.5, 0.5))
sinx_da = xr.DataArray(data=sinx, coords=coords, dims=["Time"], name="sinx")

cosx = np.cos(np.arange(0.0, coords["Time"].size * 0.5, 0.5))
cosx_da = xr.DataArray(data=cosx, coords=coords, dims=["Time"], name="cosx")

In [ ]:
ds = xr.Dataset(data_vars={"sinx": sinx_da, "cosx": cosx_da})

In [ ]:
ds

In [ ]:
print(ds)

In [ ]:
attrs = {"Description": "Sine and cosine trigonometric functions"}
ds = xr.Dataset(data_vars={"sinx": sinx, "cosx": cosx}, attrs=attrs)

In [ ]:
ds

In [ ]:
x = np.arange(0.0, 5 * np.pi, np.pi / 8)
y = x + np.pi / 2

xx, yy = np.meshgrid(x, y)

In [ ]:
xx

In [ ]:
yy

In [ ]:
plt.pcolormesh(xx)

In [ ]:
plt.pcolormesh(yy)

In [ ]:
plt.pcolormesh(np.cos(xx) * np.sin(yy))

In [ ]:
coords = {"x": x, "y": y}

da = xr.DataArray(
    data=np.cos(xx) * np.sin(yy),
    coords=coords,
    dims=["x", "y"],
    name="cosx_times_siny",
)

In [ ]:
da

In [ ]:
ds = xr.Dataset(data_vars={da.name: da})

In [ ]:
ds

In [ ]:
ds["cosx_times_siny"].plot()

In [ ]:
ds["cosx_times_siny"].mean()

In [ ]:
ds["cosx_times_siny"].mean(dim="x")

[Return to top](#top)

<a id="real-data"></a>
## 3. Working with real data

Our sine and cosine example is convenient for demonstrating how DataArrays and Datasets work. Let's try exploring some model forecast data for a real world example.

In [ ]:
ds = xr.open_dataset("data/model_forecast.nc")

In [ ]:
ds

In [ ]:
ds["temperature_2_meter"].plot()

In [ ]:
ds["temperature"].plot()

In [ ]:
ds["temperature"]

In [ ]:
ds["pressure"]

In [ ]:
ds["temperature"].sel(pressure=850.0)

In [ ]:
ds["temperature"].isel(pressure=0)

In [ ]:
ds["temperature"].sel(pressure=850.0).plot()

In [ ]:
ds["temperature_2_meter"].plot()

In [ ]:
ds["temperature_2_meter"].isel(x=200, y=200).values

In [ ]:
temp_interped = ds["temperature_2_meter"].interp(x=200.5, y=200.5).values

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ds["temperature_2_meter"].plot(ax=ax)
ax.scatter(x=200.5, y=200.5, color="black", marker="x")
ax.text(x=207, y=207, s=f"{temp_interped:.2f} {ds['temperature_2_meter'].units}")

In [ ]:
x_interp = np.arange(200.5, 300.5, 10.0)
y_interp = np.arange(200.5, 300.5, 10.0)

In [ ]:
ds["temperature_2_meter"].interp(x=x_interp, y=y_interp).values

[Return to top](#top)

<a id="exercise1"></a>
## Exercise 1

1. Select a variable that has a vertical dimension of pressure and index that variable by one of the pressure levels. Plot the data.
2. Interpolate to one or more points on the grid. Plot the data again with markers located at your interpolation point(s) and text labels describing the interpolated values.

In [ ]:
# your code here

[Return to top](#top)

<a id="scipy"></a>
## 4. Scipy for Spatial Analysis

<img src="https://docs.scipy.org/doc/scipy/_static/logo.svg" width=200/>

Scipy includes a huge number of tools for scientific computing that expands well beyond NumPy, pandas, and xarray. Here we will take just a glimpse at the `ndimage` module for analyzing spatial data.

In [ ]:
from scipy import ndimage

In [ ]:
temp_filtered = ndimage.gaussian_filter(ds["temperature_2_meter"], sigma=3.0)

fig, ax = plt.subplots(figsize=(8, 6))
ax.pcolormesh(temp_filtered)

In [ ]:
temp_filtered = ndimage.gaussian_filter(ds["temperature_2_meter"], sigma=3.0)
difference = temp_filtered - ds["temperature_2_meter"]

fig, ax = plt.subplots(figsize=(8, 6))
ax.pcolormesh(temp_filtered - ds["temperature_2_meter"], cmap="seismic", vmin=-5.0, vmax=5.0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.pcolormesh(difference[0:100, 200:300], cmap="seismic", vmin=-5.0, vmax=5.0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.pcolormesh(difference[400:, 400:500], cmap="seismic", vmin=-5.0, vmax=5.0)

In [ ]:
sobelx = ndimage.sobel(ds["temperature_2_meter"], axis=1)
sobely = ndimage.sobel(ds["temperature_2_meter"], axis=0)
sobel_filter = np.hypot(sobelx, sobely)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.pcolormesh(sobel_filter)

[Return to top](#top)

<a id="exercise2"></a>
## Exercise 2

1. Select another variable to analyze. Smooth the data using a gaussian filter and with varying sigma values. How does changing the sigma value change the smoothing of the data?